<a href="https://colab.research.google.com/github/seokwonhwang/SOE/blob/master/Lecture26_Intro_to_pytorch_geometric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pytorch_geometric이란?
----
pytorch를 기반으로 하여 graph neural network을 쉽게 만들수 있게 도와주는 라이브러리이다. 

Stanford 대학의 연구자들이 주로 만들었으며 비슷한 것으로 dgl 이라는 라이브러리도 있다. 

ref: https://pytorch-geometric.readthedocs.io/en/latest/index.html

이번 강의에서는 GPU를 이용한 hardware를 사용해보도록 하겠다. 

2021년 11월 기준의 설치 방법이나 버젼이 변화하면서 설치 방법에도 변동이 있을 수 있다는 점을 기억해주시길 바랍니다. 



In [ ]:
# Install required packages.
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import torch
import networkx as nx
import matplotlib.pyplot as plt

## Data handling of a graph

그래프는 노드사이의 관계 (링크, edge)를 모델링 하기 위해서 사용된다.

pytorch_geometric (줄여서 PyG)에서는 하나의 그래프는 **torch_geometric.data.Data** 클래스로 표현된다. 

**torch_geometric.data.Data**는 다음의 속성을 가지고 있다. 

모든 속성은 optional 하다. 

* **data.x**: 노드 특성 행렬, 다음의 차원을 가진다.[노드 개수, 노드 특성 개수]

* **data.edge_index**: 그래프의 연결 상태, [2, 연결 개수] and type torch.long

* **data.y**: 학습시키고자 하는 데이터. 여러가지 형태가 가능하다. 예를 들어, 각 노드별 예측을 수행하고 싶으면 [노드 개수, *] or 그래프 단위의 예측을 수행하고자 하면 [1, *]

* **data.edge_attr**: Edge의 특성 행렬 [연결의 개수, 연결 특성 개수]

* **data.pos**: 노드의 위치 [노드 개수, 차원]


A graph is used to model pairwise relations (edges) between objects (nodes). A single graph in PyG is described by an instance of torch_geometric.data.Data, which holds the following attributes by default:

* **data.x**: Node feature matrix with shape [num_nodes, num_node_features]

* **data.edge_index**: Graph connectivity in COO format with shape [2, num_edges] and type torch.long

* **data.edge_attr**: Edge feature matrix with shape [num_edges, num_edge_features]

* **data.y**: Target to train against (may have arbitrary shape), e.g., node-level targets of shape [num_nodes, *] or graph-level targets of shape [1, *]

* **data.pos**: Node position matrix with shape [num_nodes, num_dimensions]

None of these attributes are required. In fact, the Data object is not even restricted to these attributes. We can, e.g., extend it by data.face to save the connectivity of triangles from a 3D mesh in a tensor with shape [3, num_faces] and type torch.long.

In [ ]:
import torch
from torch_geometric.data import Data

connectivity = torch.tensor([[0, 1, 1, 2],
                             [1, 0, 2, 1]], dtype=torch.long) # 2 X 4 텐서

node_feature = torch.tensor([[-1], [0], [1]], dtype=torch.float) # 1 X 3 텐서

data = Data(x = node_feature, edge_index = connectivity)
print(data)

Data(x=[3, 1], edge_index=[2, 4])


위에서 만든 그래프는 단순한 unweighted/undirected 그래프이며 다음의 그림과 같다. 

각 노드는 1개의 특성만을 가진다. 

----

We show a simple example of an unweighted and undirected graph with three nodes and four edges. 

Each node contains exactly one feature:

<img src="https://pytorch-geometric.readthedocs.io/en/latest/_images/graph.svg" width="400"/>


연결의 인덱스는, source와 target을 정의하는 텐서, index 튜플의 리스트가 아니다. 

**모양에 주의하자. 2 X 연결 (edge) 개수**의 형태를 띄고있다. 

만일 (source, target)의 형태의 리스트로 입력하고 싶다면 transpose 와 contiguous 메소드를 다음의 예제와 같이 call 해야 한다. 

----

Note that edge_index, i.e. the tensor defining the source and target nodes of all edges, is not a list of index tuples. 

If you want to write your indices this way, you should transpose and call contiguous on it before passing them to the data constructor:



In [ ]:
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long) # 4 X 2 : 연결 개수 X 2 의 크기를 가지는 텐서. 

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())

print(data)
print(data.num_edges)
print(data.is_undirected())
print(data.is_directed())

Data(x=[3, 1], edge_index=[2, 4])
4
True
False


비록 그래프가 2개의 연결밖에 없지만 양방향을 모두 고려하기 위해서 4개의 인덱스를 지정해 주었다. 

만일 index가 하나가 빠진다면 방향성이 있는 그래프 (directed graph)로 인식된다. 

----

Although the graph has only two edges, we need to define **four index tuples to account for both directions of a edge**.

If a pair of indices are not indicated, the graph is considered as **directed**.


In [ ]:
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           #[1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())
print(data)
print(data.num_edges)
print("Is the graph undirected?", data.is_undirected())
print("Is the graph directed?", data.is_directed())

Data(x=[3, 1], edge_index=[2, 3])
3
Is the graph undirected? False
Is the graph directed? True


여러가지 노드 레벨, 연결 레벨, 그래프 레벨의 속성을 저장하기 위해서 여러가지 유틸리터 함수들을 제공한다. 

----

Besides holding a number of node-level, edge-level or graph-level attributes, Data provides a number of useful utility functions, e.g.:



In [ ]:
print(data.keys)

['x', 'edge_index']


In [ ]:
print(data['x']) # node feature를 출력하여라. 딕셔너리 사용과 유사. 

tensor([[-1.],
        [ 0.],
        [ 1.]])


* 어떤 속성이 정의되어 있는지 확인. 

* To check what attributes are defined. 

In [ ]:
for key, item in data:
    print("{} found in data".format(key))

x found in data
edge_index found in data


In [ ]:
'edge_attr' in data

False

In [ ]:
data.num_nodes

3

In [ ]:
data.num_edges

3

In [ ]:
data.num_node_features

1

In [ ]:
data.has_isolated_nodes()


True

In [ ]:
data.has_self_loops()

False

In [ ]:
# Transfer data object to GPU.
device = torch.device('cuda')
data = data.to(device)

아래 페이지에서 PyG의 Data 클래스의 전체 내용을 확인할 수 있다. 

Ref: https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data

----

You can find a complete list of all methods at https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data




# Common dataset
----


PyG는 다양한 공통적으로 사용되는 데이터셋을 이미 가지고 있습니다. 

예를 들어,  Planetoid datasets (Cora, Citeseer, Pubmed), QM7 and QM9 데이터셋, 그리고 FAUST, ModelNet10/40, ShapeNet 같은 3D point cloud 데이터들을 가지고 있습니다. 

이러한 데이터셋을 불러오는 것은 매우 직관적입니다. 

pytorch와 마찬가지로 Dataset을 자동으로 다운받고 처리될 수 있습니다. 

예를 들어서 ENZYMES 데이터셋(6개의 클래스로 분류될 수 있는 600개의 그래프)을 불러오기 위해서는 다음과 같이 할 수 있습니다. 

----

PyG contains a large number of common benchmark datasets, e.g., all Planetoid datasets (Cora, Citeseer, Pubmed), all graph classification datasets from http://graphkernels.cs.tu-dortmund.de and their cleaned versions, the QM7 and QM9 dataset, and a handful of 3D mesh/point cloud datasets like FAUST, ModelNet10/40 and ShapeNet.

Initializing a dataset is straightforward. An initialization of a dataset will automatically download its raw files and process them to the previously described Data format. E.g., to load the ENZYMES dataset (consisting of 600 graphs within 6 classes), type:

In [ ]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

In [ ]:
print(dataset)

ENZYMES(600)


In [ ]:
print(len(dataset))

600


In [ ]:
dataset.num_classes

6

In [ ]:
dataset.num_node_features

3

We now have access to all 600 graphs in the dataset:

* 첫번째 그래프를 가지고 와서 살펴보자. 

In [ ]:
data = dataset[0]
print(data)
print(data.y)

Data(edge_index=[2, 168], x=[37, 3], y=[1])
tensor([5])


* 첫번째 그래프는 37개의 노드, 168개의 edge로 구성되어 있다. 

In [ ]:
print(data.is_undirected())

True


위의 결과로 부터 첫번째 그래프는 37개의 노드를 가지고 있고, 각 노드는 3개의 특성을 가지고 있는 것을 알 수 있습니다. 

그래프에는 168/2 = 84 개의 방향성이 없는 하나의 분류에 해당하는 그래프가 존재한다. 

그리고 데이터 자체가 하나의 그래프 레벨의 예측하고자 하는 속성을 가지고 있다. 

데이터셋은 슬라이싱을 이용해서 학습용셋과 테스트셋으로 나눌수 있다. 

We can see that the first graph in the dataset contains 37 nodes, each one having 3 features. 

There are 168/2 = 84 undirected edges and the graph is assigned to exactly one class. In addition, the data object is holding exactly one graph-level target.

We can even use slices, long or bool tensors to split the dataset. E.g., to create a 90/10 train/test split, type:

In [ ]:
train_dataset = dataset[:540]
print(train_dataset)

ENZYMES(540)


In [ ]:
test_dataset = dataset[540:]
print(test_dataset)

ENZYMES(60)


데이터셋을 무작위로 섞고 싶다면 다음과 같이 할 수 있다. 

----

If you are unsure whether the dataset is already shuffled before you split, you can randomly permutate it by running:



In [ ]:
dataset = dataset.shuffle()

동일한 작업을 다음 코드를 이용해서도 할 수 있다. 

----


This is equivalent of doing:



In [ ]:
perm = torch.randperm(len(dataset))
print(perm) # 랜덤하게 섞인 index의 리스트. 

dataset = dataset[perm]
print(dataset)

tensor([424, 235, 191,  16,  19, 571, 353,  96, 158,  41, 369, 136, 253, 574,
         85, 392, 548, 350, 441,  75, 443, 439, 468, 116, 373, 144,  48, 248,
        496, 449, 209,  97, 515,  31, 285, 527, 154, 127, 526, 120, 216, 164,
        387, 258, 489, 143, 229, 330, 263, 250,  73, 265, 114,  84, 473,  63,
         88,   9, 411, 384, 245, 425, 145, 577, 122, 137,  45, 121, 375, 316,
        287, 488, 480,  26, 221, 567, 218, 510, 162, 395, 206, 368, 197, 167,
        494, 262, 354, 528,  87, 130, 241, 556, 535,  61, 518,   7,  29,  79,
        477,  93, 379, 374, 185, 300, 561, 415, 198, 214, 597,  81, 563, 364,
        456,  22, 436, 370,  27, 593, 503, 498, 125, 416, 380,  24, 587, 540,
        430, 532, 288, 179,  66, 383,  18, 393, 385, 321, 590, 572, 348, 147,
        128, 310, 156, 440, 309, 432, 386, 400, 201, 398, 207, 301, 169, 539,
        339,   2,  42, 239, 566,  92, 255, 589, 402, 378,  67, 306,  34, 583,
        172, 516, 482, 444, 219, 592, 506, 204, 132, 260, 138,  

다른 데이터셋을 받아봅시다. 

코라 데이터셋은 semi-supervised 그래프 노드 분류 테스트를 위해서 많이 사용되는 셋이다. 

-----

Let’s try another one! Let’s download Cora, the standard benchmark dataset for semi-supervised graph node classification:



In [ ]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
print(dataset)

Cora()


* Cora 데이터의 경우, 그래프 1개로 이루어져 있다. 

In [ ]:
len(dataset)

1

* 각 노드가 7개의 클래스로 분류 될 수 있다. 

In [ ]:
dataset.num_classes

7

* 하나의 노드는 1433개의 feature를 가지고 있다. 

In [ ]:
dataset.num_node_features

1433

Here, the dataset contains only a single, undirected citation graph:

In [ ]:
data = dataset[0]

In [ ]:
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [ ]:
data.is_undirected()

True

* Train_mask, val_mask, test_mask는 각각 이미 지정되어 있는 training set, validation set, test set을 의미한다. 
* 각 셋에 해당하게 되면 True 값을 가지고 그렇지 않은 경우 False 값을 가진다. 


In [ ]:
data.train_mask.sum().item()

140

In [ ]:
data.val_mask.sum().item()

500

In [ ]:
data.test_mask.sum().item()

1000

## Chemical datasets in PyG

----

PyG에는 다양한 화학 관련된 데이터셋들이 이미 존재한다. 

예를 들어, 전체 상업적으로 판매되는 분자의 데이터가 들어있는 ZINC, 양자 역학적 계산 결과가 들어있는 QM7b, QM9 등의 데이터셋이 존재한다. 

전체 데이터셋의 리스트는 다음 링크에서 찾을 수 있다. 

Ref: https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html

----

PyG includes many pre-compiled chemistry-related datasets. 

For example, ZINC dataset contains all commercially available molecules. 
QM7b and QM9 datasets have quantum mechanical properties of molecules. 

The complete list of datasets can be found in https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html




### QM9 데이터셋
----

여기에서는 간단히 QM9 데이터셋의 구성을 알아보도록 합시다. 

H, C, O, N, F 로만 구성되어 있는 분자들의 양자 계산에서 얻을 수 있는 물성 계산 값을 가지고 있다. 

QM9 데이터셋은 19가지의 양자 역학으로 계산된 분자 물성 값을 가지고 있다. 

Ref: https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.QM9

Source code: https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/datasets/qm9.html

In [ ]:
from torch_geometric.datasets import QM9

In [ ]:
dataset = QM9(root='./QM9')
print("Dataset Name:", dataset)

Dataset Name: QM9(130831)


* 전체 데이터의 개수는 130831 개임을 알 수 있다. 

In [ ]:
dataset.num_classes

19

* 19개의 예측을 위한 분자 물성 테이터를 가지고 있다. 
* 19가지의 물성은 https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.QM9 문서에서 찾을 수 있다. 

In [ ]:
dataset.num_node_features

11

* 각 노드 (원자)의 특성은 11개로 주어진다. 
* atom types = {'H': 0, 'C': 1, 'N': 2, 'O': 3, 'F': 4}
* 추가적으로 사용되는 원자의 특성은 다음과 같다: atomic_number, aromatic, sp, sp2, sp3, num_hs
* one_hot_encoding: 5 차원 + 원자의 특성 6차원 = 전체 11차원



In [ ]:
data.is_undirected()

True

In [ ]:
for key, item in data:
    print("{} found in data".format(key))

x found in data
edge_index found in data
y found in data
train_mask found in data
val_mask found in data
test_mask found in data


* 첫번째 데이터 (분자) 하나만 살펴보겠습니다. 

In [ ]:
data = dataset[0]

In [ ]:
data.y

tensor([[    0.0000,    13.2100,   -10.5499,     3.1865,    13.7363,    35.3641,
             1.2177, -1101.4878, -1101.4098, -1101.3840, -1102.0229,     6.4690,
           -17.1722,   -17.2868,   -17.3897,   -16.1519,   157.7118,   157.7100,
           157.7070]])

In [ ]:
print(data)

Data(x=[5, 11], edge_index=[2, 8], edge_attr=[8, 4], y=[1, 19], pos=[5, 3], idx=[1], name='gdb_1', z=[5])


In [ ]:
data.x

tensor([[0., 1., 0., 0., 0., 6., 0., 0., 0., 0., 4.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])